In [2]:
# Must contain this cell, in order to import from other folders
import os
import sys
sys.path.append(os.getcwd() + '/..')

%reload_ext autoreload
%autoreload 2

In [4]:
import torch
import pytorch_lightning as pl

from src.datamodules.age.datamodule import AgeDataModule
from src.models.module import PLModule
from src.models.fe import FeatureExtractor

# Prepare models
backbone = FeatureExtractor(backbone_name='vgg16', output_size=8)
model = PLModule(backbone)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
model.set_optimizer(optimizer)

# Prepare datamodule
params = {
    'fold': 0,
    'batch_size': 32,
    'image_width': 224,
    'image_height': 224
}
datamodule = AgeDataModule(**params)

trainer = pl.Trainer(fast_dev_run=True)
trainer.fit(model=model, datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Running in fast_dev_run mode: will run a full train, val, test and prediction loop using 1 batch(es).

  | Name  | Type             | Params
-------------------------------------------
0 | model | FeatureExtractor | 134 M 
-------------------------------------------
32.8 K    Trainable params
134 M     Non-trainable params
134 M     Total params
537.173   Total estimated model params size (MB)
/Users/dudulasri/miniconda3/envs/face-age-detector/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:117: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/Users/dudulasri/mi

Epoch 0: 100%|██████████| 2/2 [00:20<00:00, 10.07s/it, loss=2.12, v_num=]


In [5]:
trainer.callback_metrics

{'train/precision': tensor(0.0273),
 'train/recall': tensor(0.0312),
 'train/f1-score': tensor(0.0292),
 'train/loss': tensor(2.1236),
 'valid/precision': tensor(0.9375),
 'valid/recall': tensor(0.0312),
 'valid/f1-score': tensor(0.0605),
 'valid/loss': tensor(1.9823)}

In [64]:
datamodule.setup()
train_dataloader = datamodule.train_dataloader()
images, labels = next(iter(train_dataloader))

In [69]:
outputs = model(images)

In [75]:
predictions = outputs.argmax(dim=1)
predictions

tensor([3, 7, 6, 6, 7, 5, 5, 4, 3, 5, 6, 5, 5, 5, 6, 5, 5, 1, 2, 5, 2, 6, 5, 1,
        6, 3, 7, 0, 5, 6, 5, 7])

In [85]:
from src.datamodules.age.constants import AGE_LABELS
from sklearn.metrics import classification_report

report = classification_report(y_true=labels, y_pred=predictions, target_names=AGE_LABELS, output_dict=True)

/Users/dudulasri/miniconda3/envs/face-age-detector/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


NameError: name 'trainer' is not defined

In [70]:
import torch.nn.functional as F

F.cross_entropy(outputs, labels)

tensor(2.2203, grad_fn=<NllLossBackward0>)

In [72]:
import os

os.cpu_count()

16